# Capstone Project

### Segmenting and Clustering Neighborhoods in Toronto
#### Week 3, part 3

### Importing libraries

In [1]:
import numpy as np
import pandas as pd

import requests

import folium
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

### Scraping the data from Wikipedia

In [2]:
df = pd.read_csv('df_week3_part2.csv')
df.head(3)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.749975,-79.327600
1,M4A,North York,Victoria Village,43.730638,-79.312965
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640


## Toronto map

In [3]:
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto-explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto coordinates {}, {}.'.format(latitude, longitude))

Toronto coordinates 43.6534817, -79.3839347.


In [4]:
df_toronto = df[df['Borough'].str.contains("Toronto")].reset_index(drop=True)
df_toronto.shape

(39, 5)

In [5]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='grey',
        fill=True,
        fill_color='grey',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

## Exploring neighborhoods with Foursquare API

In [6]:
CLIENT_ID = '' # your Foursquare ID
CLIENT_SECRET = '' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: GCL10AE55OVRG4KYFKO42QCAL451RC0UDPO2DKTSPJDPNEZR
CLIENT_SECRET:V55HVDVL0PEIBTNS0U0SVIEEKEA2EGYMNF4THP13CJIY22CE


### Retreiving venues

In [7]:
def getNearbyVenues(boroughs, neighborhoods, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for borough, neighborhood, lat, lng in zip(boroughs, neighborhoods, latitudes, longitudes):
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            borough,
            neighborhood, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = [
                  'Borough',
                  'Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [8]:
venues = getNearbyVenues(boroughs=df_toronto['Borough'],
                                             neighborhoods=df_toronto['Neighborhood'],
                                             latitudes=df_toronto['Latitude'],
                                             longitudes=df_toronto['Longitude'])

In [9]:
venues.head(3)

,Borough,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Roselle Desserts,43.653447,-79.362017,Bakery
1,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,Figs Breakfast & Lunch,43.655675,-79.364503,Breakfast Spot


In [10]:
venues.shape

(1717, 8)

### Analysing neighbourhoods fo the boroughs

In [11]:
venues.groupby(['Borough', 'Neighborhood']).count()

Neighborhood Latitude  ...  Venue Category
Borough                Neighborhood                                                               ...                
Central Toronto        Davisville                                                             26  ...              26
                       Davisville North                                                       10  ...              10
                       Forest Hill North & West                                                1  ...               1
                       Lawrence Park                                                           2  ...               2
                       Moore Park, Summerhill East                                             3  ...               3
                       North Toronto West                                                      4  ...               4
                       Roselawn                                                                1  ...               1
                       Summerhill West, Rathnelly, South Hill, Forest ...                      6  ...               6
                       The Annex, North Midtown, Yorkville                                    22  ...              22
Downtown Toronto       Berczy Park                                                            63  ...              63
                       CN Tower, King and Spadina, Railway Lands, Harb...                     81  ...              81
                       Central Bay Street                                                     71  ...              71
                       Christie                                                               10  ...              10
                       Church and Wellesley                                                   77  ...              77
                       Commerce Court, Victoria Hotel                                        100  ...             100
                       First Canadian Place, Underground city                                100  ...             100
                       Garden District, Ryerson                                              100  ...             100
                       Harbourfront East, Union Station, Toronto Islands                      63  ...              63
                       Kensington Market, Chinatown, Grange Park                              55  ...              55
                       Regent Park, Harbourfront                                              24  ...              24
                       Richmond, Adelaide, King                                              100  ...             100
                       Rosedale                                                                4  ...               4
                       St. James Town                                                         86  ...              86
                       St. James Town, Cabbagetown                                            44  ...              44
                       Toronto Dominion Centre, Design Exchange                              100  ...             100
                       University of Toronto, Harbord                                         50  ...              50
Downtown Toronto Stn A Enclave of M5E                                                        100  ...             100
East Toronto           India Bazaar, The Beaches West                                         20  ...              20
                       Studio District                                                        49  ...              49
                       The Beaches                                                             5  ...               5
                       The Danforth West, Riverdale                                            9  ...               9
East Toronto Business  Enclave of M4L                                                        100  ...             100
East York/East Toronto The Danforth  East                      

### Finding unique categories

In [12]:

print(f'There is {venues["Venue Category"].nunique()} uniques categories.')

There is 221 uniques categories.


In [13]:
# one hot encoding
toronto_onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Borough'] = venues['Borough'] 
toronto_onehot['Neighborhoods'] = venues['Neighborhood'] 
toronto_onehot['Venue'] = venues['Venue'] 

# move neighborhood column to the first column
fixed_columns = list(toronto_onehot.columns[-3:]) + list(toronto_onehot.columns[:-3])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head(3)

,Borough,Neighborhoods,Venue,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Downtown Toronto,"Regent Park, Harbourfront",Roselle Desserts,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Downtown Toronto,"Regent Park, Harbourfront",Tandem Coffee,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Downtown Toronto,"Regent Park, Harbourfront",Figs Breakfast & Lunch,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [14]:
toronto_onehot.shape

(1717, 224)

### Grouping rows by borough and neighborhood and by taking the mean of the frequency of occurrence of each category

In [15]:
toronto_grouped = toronto_onehot.groupby(['Borough', 'Neighborhoods']).mean().reset_index()
toronto_grouped

,Borough,Neighborhoods,American Restaurant,Antique Shop,Aquarium,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bank,Bar,Basketball Stadium,Beer Bar,Beer Store,Belgian Restaurant,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burrito Place,Bus Line,Butcher,Café,Candy Store,Caribbean Restaurant,Cheese Shop,Chinese Restaurant,Chocolate Shop,...,Sculpture Garden,Seafood Restaurant,Shoe Store,Shopping Mall,Skating Rink,Smoke Shop,Snack Place,Soccer Field,Soup Place,Souvlaki Shop,Spa,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Strip Club,Summer Camp,Supermarket,Sushi Restaurant,Swim School,Taco Place,Tailor Shop,Taiwanese Restaurant,Tanning Salon,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Wings Joint,Yoga Studio
0,Central Toronto,Davisville,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.076923,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.038462,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038462,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.038462,0.000000,0.000000,0.000000,0.038462,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
1,Central Toronto,Davisville North,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.100000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
2,Central Toronto,Forest Hill North & West,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
3,Central Toronto,Lawrence Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.5,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.5,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000
4,Central Toronto,"Moore Park, Summerhill East",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0000

In [16]:
toronto_grouped.shape

(39, 223)

### Printing each neighborhood along with the top 5 most common venues

In [17]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhoods']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhoods'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[2:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Davisville----
                venue  freq
0        Dessert Shop  0.12
1      Sandwich Place  0.08
2         Pizza Place  0.08
3  Italian Restaurant  0.08
4         Coffee Shop  0.08


----Davisville North----
               venue  freq
0   Department Store   0.1
1              Hotel   0.1
2        Pizza Place   0.1
3         Playground   0.1
4  Food & Drink Shop   0.1


----Forest Hill North & West----
                             venue  freq
0                             Park   1.0
1              American Restaurant   0.0
2                           Office   0.0
3  Molecular Gastronomy Restaurant   0.0
4              Monument / Landmark   0.0


----Lawrence Park----
                             venue  freq
0                         Bus Line   0.5
1                      Swim School   0.5
2               Miscellaneous Shop   0.0
3       Modern European Restaurant   0.0
4  Molecular Gastronomy Restaurant   0.0


----Moore Park, Summerhill East----
                 venue  freq
0    C

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Arranging the results into a dataframe

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[2:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Borough', 'Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Borough'] = toronto_grouped['Borough']
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhoods']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 2:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head(3)

,Borough,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Central Toronto,Davisville,Dessert Shop,Italian Restaurant,Sandwich Place,Pizza Place,Coffee Shop,Café,Indoor Play Area,Seafood Restaurant,Farmers Market,Park
1,Central Toronto,Davisville North,Pizza Place,Japanese Restaurant,Playground,Department Store,Park,Sandwich Place,Breakfast Spot,Food & Drink Shop,Hotel,Gym / Fitness Center
2,Central Toronto,Forest Hill North & West,Park,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market


In [20]:
neighborhoods_venues_sorted.shape

(39, 12)

### Clustering

In [21]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop(['Borough', 'Neighborhoods'], 1)

# run k-means clustering
kmeans = KMeans(init='k-means++', n_clusters=kclusters, random_state=1, max_iter=1000).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([0, 0, 1, 4, 0, 0, 2, 0, 0, 0], dtype=int32)

In [22]:
# add clustering labels
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_

toronto_merged = df_toronto.copy()

# merge toronto_grouped with df_toronto_borough to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.drop(['Borough'], axis=1).set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head(3) # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.655120,-79.362640,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Distribution Center,Playground,Pub,Restaurant,Electronics Store,Event Space,0
1,M5B,Downtown Toronto,"Garden District, Ryerson",43.657491,-79.378495,Coffee Shop,Clothing Store,Café,Burger Joint,Cosmetics Shop,Hotel,Japanese Restaurant,Diner,Bookstore,Tea Room,0
2,M5C,Downtown Toronto,St. James Town,43.651900,-79.375736,Coffee Shop,Restaurant,Café,Seafood Restaurant,Hotel,Cosmetics Shop,Cocktail Bar,Italian Restaurant,Department Store,Farmers Market,0


In [23]:
toronto_merged.tail(3)

,PostalCode,Borough,Neighborhood,Latitude,Longitude,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
36,M5X,Downtown Toronto,"First Canadian Place, Underground city",43.647975,-79.381445,Coffee Shop,Hotel,Café,Restaurant,Gym,American Restaurant,Seafood Restaurant,Deli / Bodega,Japanese Restaurant,Concert Hall,0
37,M4Y,Downtown Toronto,Church and Wellesley,43.666585,-79.381330,Coffee Shop,Japanese Restaurant,Sushi Restaurant,Gay Bar,Restaurant,Café,Dance Studio,Pub,Bubble Tea Shop,Hotel,0
38,M7Y,East Toronto Business,Enclave of M4L,43.648690,-79.385440,Coffee Shop,Hotel,Restaurant,Asian Restaurant,Bar,Italian Restaurant,Mediterranean Restaurant,Movie Theater,Fast Food Restaurant,Taco Place,0


### Visualizing the clusters

In [24]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examining the clusters

#### Cluster 1

In [25]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
0,Downtown Toronto,Coffee Shop,Breakfast Spot,Yoga Studio,Bakery,Distribution Center,Playground,Pub,Restaurant,Electronics Store,Event Space,0
1,Downtown Toronto,Coffee Shop,Clothing Store,Café,Burger Joint,Cosmetics Shop,Hotel,Japanese Restaurant,Diner,Bookstore,Tea Room,0
2,Downtown Toronto,Coffee Shop,Restaurant,Café,Seafood Restaurant,Hotel,Cosmetics Shop,Cocktail Bar,Italian Restaurant,Department Store,Farmers Market,0
3,East Toronto,Flower Shop,Health Food Store,Trail,Pub,Neighborhood,Eastern European Restaurant,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,0
4,Downtown Toronto,Coffee Shop,Seafood Restaurant,American Restaurant,Cheese Shop,Pharmacy,Hotel,Farmers Market,Restaurant,Cocktail Bar,Pub,0
5,Downtown Toronto,Coffee Shop,Clothing Store,Sandwich Place,Hotel,Plaza,Cosmetics Shop,Middle Eastern Restaurant,Bookstore,Gym / Fitness Center,Ramen Restaurant,0
6,Downtown Toronto,Café,Grocery Store,Coffee Shop,Playground,Candy Store,Baby Store,Ethiopian Restaurant,Flower Shop,Flea Market,Fish Market,0
7,Downtown Toronto,Café,Coffee Shop,Hotel,Gym,Restaurant,Steakhouse,Japanese Restaurant,Asian Restaurant,Seafood Restaurant,Concert Hall,0
8,West Toronto,Park,Bakery,Café,Fast Food Restaurant,Liquor Store,Coffee Shop,Post Office,Bank,Smoke Shop,Pet Store,0
10,Downtown Toronto,Coffee Shop,Hotel,Restaurant,Japanese Restaurant,Boat or Ferry,Park,Plaza,Supermarket,Aquarium,Sporting Goods Shop,0


#### Cluster 2

In [26]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
21,Central Toronto,Park,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,1


#### Cluster 3

In [27]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
19,Central Toronto,Home Service,Food Court,Food & Drink Shop,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,2


#### Cluster 4

In [28]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
9,East York/East Toronto,Convenience Store,Park,Yoga Studio,Electronics Store,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,3
22,West Toronto,Convenience Store,Park,Residential Building (Apartment / Condo),Yoga Studio,Electronics Store,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,3


#### Cluster 5

In [29]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,Cluster Labels
18,Central Toronto,Bus Line,Swim School,Yoga Studio,Food Court,Food,Flower Shop,Flea Market,Fish Market,Fish & Chips Shop,Fast Food Restaurant,4


In [30]:
toronto_merged.to_csv('df_week3_part3.csv', index=False)